In [ ]:
import asyncio
from langchain_ollama import ChatOllama
from queue import Queue
import re

class AsyncSentenceQueue:
    def __init__(self):
        self.queue = asyncio.Queue()
        self.current_sentence = ""
        
    async def put(self, text: str):
        """Add text and split into sentences when possible"""
        self.current_sentence += text
        sentences = re.split(r'([.!?]+)', self.current_sentence)
        
        # Process complete sentences
        while len(sentences) >= 2:  # Need both sentence content and separator
            sentence = sentences.pop(0) + sentences.pop(0)  # Combine with separator
            if sentence.strip():  # Only queue non-empty sentences
                await self.queue.put(sentence)
        
        # Store remaining incomplete sentence
        self.current_sentence = ''.join(sentences)
    
    async def get(self):
        """Get next complete sentence from queue"""
        return await self.queue.get()
    
    def task_done(self):
        """Mark a queue item as done"""
        self.queue.task_done()
    
    async def finish(self):
        """Put any remaining text into queue"""
        if self.current_sentence.strip():
            await self.queue.put(self.current_sentence)
            self.current_sentence = ""

async def generate_text(text: str, sentence_queue: AsyncSentenceQueue):
    """Generate text and put sentences into queue"""
    model = ChatOllama(model="llama3.2:1b")
    try:
        stream = model.stream(text)
        for chunk in stream:
            await sentence_queue.put(chunk.content)
            await asyncio.sleep(0)  # Yield control
        await sentence_queue.finish()  # Queue any remaining text
    except asyncio.CancelledError:
        print("\nOutput generation cancelled.")
        raise

async def display_queue(sentence_queue: AsyncSentenceQueue):
    """Display sentences from queue with delay for effect"""
    try:
        while True:
            sentence = await sentence_queue.get()
            print(sentence, end='', flush=True)
            await asyncio.sleep(0.5)  # Artificial delay for queuing effect
            sentence_queue.task_done()
    except asyncio.CancelledError:
        print("\nDisplay task cancelled.")
        raise

async def main():
    sentence_queue = AsyncSentenceQueue()
    
    # Create tasks
    generator_task = asyncio.create_task(generate_text("tell me a story", sentence_queue))
    display_task = asyncio.create_task(display_queue(sentence_queue))
    
    # Wait for 5 seconds before cancelling
    try:
        await asyncio.sleep(5)
        print("\nCancelling tasks...")
        generator_task.cancel()
        display_task.cancel()
        await asyncio.gather(generator_task, display_task, return_exceptions=True)
    except asyncio.CancelledError:
        print("Main task cancelled.")
# Run the program
await main()

In [ ]:
import speech_recognition as sr
from gtts import gTTS
from io import BytesIO
from pydub import AudioSegment
from pydub.playback import play
from langchain_ollama import ChatOllama
import threading
import queue
import logging
import time
import re
import asyncio
import pyaudio
import audioop

class VoiceChatSystem:
    def __init__(self, lang='en', model_name="llama2:7b"):
        # Initialize logging
        self.logger = self._setup_logger()
        
        # Speech recognition settings
        self.recognizer = sr.Recognizer()
        self.lang = lang
        self.energy_threshold = 1000
        self.pause_threshold = 0.8
        self.min_speech_duration = 0.3
        
        # Initialize queues
        self.text_queue = queue.Queue()  # For recognized speech
        self.sentence_queue = queue.Queue()  # For generated sentences
        self.speaking_queue = queue.Queue()  # For text to be spoken
        
        # Initialize flags
        self.is_listening = True
        self.stop_current_task = False
        self.is_speaking = False
        
        # Initialize model
        self.model = ChatOllama(model=model_name)
        
        # Initialize threads
        self.listen_thread = None
        self.process_thread = None
        self.speak_thread = None
        
    def _setup_logger(self):
        logger = logging.getLogger('VoiceChatSystem')
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)
        return logger

    def speak_text(self, text):
        """Convert text to speech and play it"""
        try:
            mp3_fp = BytesIO()
            tts = gTTS(text=text, lang=self.lang)
            tts.write_to_fp(mp3_fp)
            mp3_fp.seek(0)
            audio = AudioSegment.from_mp3(mp3_fp)
            play(audio)
        finally:
            mp3_fp.close()

    def listen_continuously(self):
        """Continuously listen for speech and add to text queue"""
        with sr.Microphone() as source:
            self.recognizer.adjust_for_ambient_noise(source, duration=1)
            
            while self.is_listening:
                try:
                    self.logger.info("Listening...")
                    audio = self.recognizer.listen(source)
                    text = self.recognizer.recognize_google(audio, language=self.lang)
                    
                    if text.lower() in ['quit', 'exit', 'stop', 'bye']:
                        self.stop_system()
                        break
                    
                    self.logger.info(f"Recognized: {text}")
                    self.stop_current_task = True  # Signal to stop current processing
                    
                    # Clear existing queues
                    self._clear_queues()
                    
                    # Add new text to queue
                    self.text_queue.put(text)
                    
                except sr.UnknownValueError:
                    continue
                except sr.RequestError as e:
                    self.logger.error(f"Could not request results: {str(e)}")
                    continue

    def process_text(self):
        """Process recognized text and generate responses"""
        while self.is_listening:
            try:
                text = self.text_queue.get(timeout=1)
                self.stop_current_task = False
                
                # Generate response using the model
                stream = self.model.stream(text)
                current_sentence = ""
                
                for chunk in stream:
                    if self.stop_current_task:
                        break
                    
                    current_sentence += chunk.content
                    sentences = re.split(r'([.!?]+)', current_sentence)
                    
                    # Process complete sentences
                    while len(sentences) >= 2:
                        sentence = sentences.pop(0) + sentences.pop(0)
                        if sentence.strip():
                            self.sentence_queue.put(sentence)
                    
                    current_sentence = ''.join(sentences)
                
                # Add any remaining text
                if current_sentence.strip() and not self.stop_current_task:
                    self.sentence_queue.put(current_sentence)
                
            except queue.Empty:
                continue
            except Exception as e:
                self.logger.error(f"Error in processing: {str(e)}")

    def speak_responses(self):
        """Speak generated responses from the sentence queue"""
        while self.is_listening:
            try:
                sentence = self.sentence_queue.get(timeout=1)
                if not self.stop_current_task:
                    self.is_speaking = True
                    self.speak_text(sentence)
                    self.is_speaking = False
            except queue.Empty:
                continue
            except Exception as e:
                self.logger.error(f"Error in speaking: {str(e)}")

    def _clear_queues(self):
        """Clear all queues"""
        while not self.text_queue.empty():
            self.text_queue.get()
        while not self.sentence_queue.empty():
            self.sentence_queue.get()
        while not self.speaking_queue.empty():
            self.speaking_queue.get()

    def stop_system(self):
        """Stop all threads and clean up"""
        self.is_listening = False
        self.stop_current_task = True
        self._clear_queues()

    def start(self):
        """Start all threads"""
        self.listen_thread = threading.Thread(target=self.listen_continuously)
        self.process_thread = threading.Thread(target=self.process_text)
        self.speak_thread = threading.Thread(target=self.speak_responses)
        
        self.listen_thread.start()
        self.process_thread.start()
        self.speak_thread.start()
        
        try:
            # Keep main thread alive until stop signal
            while self.is_listening:
                time.sleep(0.1)
        except KeyboardInterrupt:
            self.logger.info("Stopping system...")
            self.stop_system()
        
        # Wait for threads to finish
        self.listen_thread.join()
        self.process_thread.join()
        self.speak_thread.join()

def main():
    system = VoiceChatSystem(lang='en', model_name="llama3.2:1b")
    system.start()

if __name__ == "__main__":
    main()

In [ ]:
import speech_recognition as sr
from gtts import gTTS
from io import BytesIO
from pydub import AudioSegment
from pydub.playback import play
from langchain_ollama import ChatOllama
import threading
import queue
import logging
import time
import re
import signal
import pygame  # For immediate audio control
import os
from threading import Event

class VoiceChatSystem:
    def __init__(self, lang='en', model_name="llama2:7b"):
        # Initialize logging
        self.logger = self._setup_logger()
        
        # Speech recognition settings
        self.recognizer = sr.Recognizer()
        self.lang = lang
        self.energy_threshold = 1000
        
        # Initialize queues with priority queue for immediate interruption
        self.text_queue = queue.Queue()
        self.sentence_queue = queue.Queue()
        
        # Initialize flags and events
        self.is_listening = True
        self.current_task_id = 0
        self.interrupt_event = Event()
        self.speaking_event = Event()
        
        # Initialize pygame mixer for immediate audio control
        pygame.mixer.init()
        
        # Initialize model
        self.model = ChatOllama(model=model_name)
        
        # Initialize threads
        self.listen_thread = None
        self.process_thread = None
        self.speak_thread = None
        
        # Set up signal handling for immediate interruption
        signal.signal(signal.SIGINT, self.signal_handler)

    def _setup_logger(self):
        logger = logging.getLogger('VoiceChatSystem')
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)
        return logger

    def signal_handler(self, signum, frame):
        self.stop_system()

    def immediate_interrupt(self):
        """Immediately interrupt all ongoing processes"""
        self.interrupt_event.set()
        if pygame.mixer.get_busy():
            pygame.mixer.stop()
        self._clear_queues()
        self.interrupt_event.clear()

    def speak_text(self, text, task_id):
        """Convert text to speech and play it with immediate interrupt capability"""
        if task_id != self.current_task_id:
            return False
        
        try:
            if self.interrupt_event.is_set():
                return False
                
            self.speaking_event.set()
            mp3_fp = BytesIO()
            tts = gTTS(text=text, lang=self.lang)
            tts.write_to_fp(mp3_fp)
            mp3_fp.seek(0)
            
            # Save temporary file for pygame
            temp_file = f"temp_audio_{task_id}.mp3"
            with open(temp_file, 'wb') as f:
                f.write(mp3_fp.getvalue())
            
            # Play with pygame for immediate interrupt capability
            pygame.mixer.music.load(temp_file)
            pygame.mixer.music.play()
            
            # Wait for audio to finish or interruption
            while pygame.mixer.music.get_busy() and not self.interrupt_event.is_set():
                time.sleep(0.1)
                
            pygame.mixer.music.stop()
            
            # Clean up
            os.remove(temp_file)
            return not self.interrupt_event.is_set()
            
        finally:
            self.speaking_event.clear()
            mp3_fp.close()

    def listen_continuously(self):
        """Continuously listen for speech with immediate interruption"""
        with sr.Microphone() as source:
            self.recognizer.adjust_for_ambient_noise(source, duration=1)
            
            while self.is_listening:
                try:
                    self.logger.info("Listening...")
                    audio = self.recognizer.listen(source, phrase_time_limit=None)
                    
                    # Immediate interrupt on new audio
                    self.immediate_interrupt()
                    
                    text = self.recognizer.recognize_google(audio, language=self.lang)
                    
                    if text.lower() in ['quit', 'exit', 'stop', 'bye']:
                        self.stop_system()
                        break
                    
                    self.logger.info(f"Recognized: {text}")
                    
                    # Update task ID and queue new text
                    self.current_task_id += 1
                    self.text_queue.put((text, self.current_task_id))
                    
                except sr.UnknownValueError:
                    continue
                except sr.RequestError as e:
                    self.logger.error(f"Could not request results: {str(e)}")
                    continue

    def process_text(self):
        """Process recognized text with immediate interrupt capability"""
        while self.is_listening:
            try:
                text, task_id = self.text_queue.get(timeout=1)
                
                if self.interrupt_event.is_set():
                    continue
                
                current_sentence = ""
                stream = self.model.stream(text)
                
                for chunk in stream:
                    if self.interrupt_event.is_set() or task_id != self.current_task_id:
                        break
                    
                    current_sentence += chunk.content
                    sentences = re.split(r'([.!?]+)', current_sentence)
                    
                    while len(sentences) >= 2 and not self.interrupt_event.is_set():
                        sentence = sentences.pop(0) + sentences.pop(0)
                        if sentence.strip():
                            self.sentence_queue.put((sentence, task_id))
                    
                    current_sentence = ''.join(sentences)
                
                if current_sentence.strip() and not self.interrupt_event.is_set():
                    self.sentence_queue.put((current_sentence, task_id))
                
            except queue.Empty:
                continue
            except Exception as e:
                self.logger.error(f"Error in processing: {str(e)}")

    def speak_responses(self):
        """Speak responses with immediate interrupt capability"""
        while self.is_listening:
            try:
                if self.interrupt_event.is_set():
                    continue
                    
                sentence, task_id = self.sentence_queue.get(timeout=1)
                
                if task_id == self.current_task_id and not self.interrupt_event.is_set():
                    self.speak_text(sentence, task_id)
                
            except queue.Empty:
                continue
            except Exception as e:
                self.logger.error(f"Error in speaking: {str(e)}")

    def _clear_queues(self):
        """Clear all queues immediately"""
        while not self.text_queue.empty():
            try:
                self.text_queue.get_nowait()
            except queue.Empty:
                break
                
        while not self.sentence_queue.empty():
            try:
                self.sentence_queue.get_nowait()
            except queue.Empty:
                break

    def stop_system(self):
        """Stop all processes immediately"""
        self.is_listening = False
        self.immediate_interrupt()
        pygame.mixer.quit()

    def start(self):
        """Start the system with all threads"""
        self.listen_thread = threading.Thread(target=self.listen_continuously)
        self.process_thread = threading.Thread(target=self.process_text)
        self.speak_thread = threading.Thread(target=self.speak_responses)
        
        self.listen_thread.start()
        self.process_thread.start()
        self.speak_thread.start()
        
        try:
            while self.is_listening:
                time.sleep(0.1)
        except KeyboardInterrupt:
            self.stop_system()
        
        self.listen_thread.join()
        self.process_thread.join()
        self.speak_thread.join()

def main():
    system = VoiceChatSystem(lang='en', model_name="llama3.2:1b")
    system.start()

if __name__ == "__main__":
    main()

In [ ]:
import speech_recognition as sr
from gtts import gTTS
from io import BytesIO
import pygame
import threading
import queue
import logging
import time
import re
import signal
import os
import numpy as np
from threading import Event
from langchain_ollama import ChatOllama
import webrtcvad  # For voice activity detection

class VoiceChatSystem:
    def __init__(self, lang='en', model_name="llama3.2:1b"):
        # Initialize logging
        self.logger = self._setup_logger()
        
        # Speech recognition settings
        self.recognizer = sr.Recognizer()
        self.lang = lang
        self.energy_threshold = 1000
        
        # Initialize queues
        self.text_queue = queue.Queue()
        self.sentence_queue = queue.Queue()
        
        # Initialize flags and events
        self.is_listening = True
        self.current_task_id = 0
        self.interrupt_event = Event()
        self.speaking_event = Event()
        self.is_system_speaking = False  # Flag to track system speech
        
        # Initialize audio control
        pygame.mixer.init(frequency=24000)
        
        # Initialize VAD for speech detection
        self.vad = webrtcvad.Vad(3)  # Aggressiveness level 3 (maximum)
        
        # Initialize model
        self.model = ChatOllama(model=model_name)
        
        # Keep track of active threads for complete cleanup
        self.active_threads = set()
        self.thread_lock = threading.Lock()
        
    def _setup_logger(self):
        logger = logging.getLogger('VoiceChatSystem')
        handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)
        return logger

    def kill_thread(self, thread_id):
        """Remove thread from active threads"""
        with self.thread_lock:
            self.active_threads.discard(thread_id)

    def register_thread(self, thread_id):
        """Register new thread"""
        with self.thread_lock:
            self.active_threads.add(thread_id)

    def immediate_interrupt(self):
        """Completely stop all ongoing processes"""
        self.interrupt_event.set()
        
        # Stop audio
        if pygame.mixer.get_busy():
            pygame.mixer.stop()
        
        # Clear all queues
        self._clear_queues()
        
        # Kill all active threads except the current one
        current_thread_id = threading.get_ident()
        with self.thread_lock:
            threads_to_kill = self.active_threads.copy()
            threads_to_kill.discard(current_thread_id)
            self.active_threads.clear()
            self.active_threads.add(current_thread_id)
        
        self.interrupt_event.clear()
        self.is_system_speaking = False

    def speak_text(self, text, task_id):
        """Convert text to speech with system speech tracking"""
        if task_id != self.current_task_id:
            return False
        
        try:
            if self.interrupt_event.is_set():
                return False
                
            self.is_system_speaking = True  # Mark system as speaking
            self.speaking_event.set()
            
            mp3_fp = BytesIO()
            tts = gTTS(text=text, lang=self.lang)
            tts.write_to_fp(mp3_fp)
            mp3_fp.seek(0)
            
            temp_file = f"temp_audio_{task_id}.mp3"
            with open(temp_file, 'wb') as f:
                f.write(mp3_fp.getvalue())
            
            pygame.mixer.music.load(temp_file)
            pygame.mixer.music.play()
            
            while pygame.mixer.music.get_busy() and not self.interrupt_event.is_set():
                time.sleep(0.1)
            
            pygame.mixer.music.stop()
            os.remove(temp_file)
            
            return not self.interrupt_event.is_set()
            
        finally:
            self.is_system_speaking = False  # Mark system as not speaking
            self.speaking_event.clear()
            mp3_fp.close()

    def is_valid_human_speech(self, audio_data):
        """Check if audio is valid human speech and not system output"""
        if self.is_system_speaking:
            return False
            
        try:
            # Convert audio data to the format needed by WebRTC VAD
            raw_data = np.frombuffer(audio_data.frame_data, dtype=np.int16)
            
            # Split audio into frames and check each frame
            frame_duration = 30  # ms
            frames = len(raw_data) // (16000 * frame_duration // 1000)
            
            speech_frames = 0
            for i in range(frames):
                start = i * (16000 * frame_duration // 1000)
                end = start + (16000 * frame_duration // 1000)
                frame = raw_data[start:end].tobytes()
                
                if self.vad.is_speech(frame, 16000):
                    speech_frames += 1
            
            # Require at least 30% of frames to contain speech
            return speech_frames / frames > 0.3
            
        except Exception as e:
            self.logger.error(f"Error in speech validation: {str(e)}")
            return False

    def listen_continuously(self):
        """Listen for speech with self-speech filtering"""
        thread_id = threading.get_ident()
        self.register_thread(thread_id)
        
        with sr.Microphone() as source:
            self.recognizer.adjust_for_ambient_noise(source, duration=1)
            
            while self.is_listening:
                try:
                    self.logger.info("Listening...")
                    audio = self.recognizer.listen(source, phrase_time_limit=None)
                    
                    # Skip if system is speaking or audio is not valid human speech
                    if self.is_system_speaking or not self.is_valid_human_speech(audio):
                        continue
                    
                    # Valid human speech detected - interrupt everything
                    self.immediate_interrupt()
                    
                    text = self.recognizer.recognize_google(audio, language=self.lang)
                    
                    if text.lower() in ['quit', 'exit', 'stop', 'bye']:
                        self.stop_system()
                        break
                    
                    self.logger.info(f"Recognized human speech: {text}")
                    
                    # New task
                    self.current_task_id += 1
                    self.text_queue.put((text, self.current_task_id))
                    
                except sr.UnknownValueError:
                    continue
                except sr.RequestError as e:
                    self.logger.error(f"Could not request results: {str(e)}")
                    continue
                
        self.kill_thread(thread_id)

    def process_text(self):
        """Process text with complete task isolation"""
        thread_id = threading.get_ident()
        self.register_thread(thread_id)
        
        while self.is_listening:
            try:
                text, task_id = self.text_queue.get(timeout=1)
                
                if self.interrupt_event.is_set():
                    continue
                
                current_sentence = ""
                stream = self.model.stream(text)
                
                for chunk in stream:
                    if self.interrupt_event.is_set() or task_id != self.current_task_id:
                        break
                    
                    current_sentence += chunk.content
                    sentences = re.split(r'([.!?]+)', current_sentence)
                    
                    while len(sentences) >= 2 and not self.interrupt_event.is_set():
                        sentence = sentences.pop(0) + sentences.pop(0)
                        if sentence.strip():
                            self.sentence_queue.put((sentence, task_id))
                    
                    current_sentence = ''.join(sentences)
                
                if current_sentence.strip() and not self.interrupt_event.is_set():
                    self.sentence_queue.put((current_sentence, task_id))
                
            except queue.Empty:
                continue
            except Exception as e:
                self.logger.error(f"Error in processing: {str(e)}")
        
        self.kill_thread(thread_id)

    def speak_responses(self):
        """Speak responses with complete task isolation"""
        thread_id = threading.get_ident()
        self.register_thread(thread_id)
        
        while self.is_listening:
            try:
                if self.interrupt_event.is_set():
                    continue
                    
                sentence, task_id = self.sentence_queue.get(timeout=1)
                
                if task_id == self.current_task_id and not self.interrupt_event.is_set():
                    self.speak_text(sentence, task_id)
                
            except queue.Empty:
                continue
            except Exception as e:
                self.logger.error(f"Error in speaking: {str(e)}")
        
        self.kill_thread(thread_id)

    def _clear_queues(self):
        """Completely clear all queues"""
        # Clear text queue
        while True:
            try:
                self.text_queue.get_nowait()
            except queue.Empty:
                break
        
        # Clear sentence queue
        while True:
            try:
                self.sentence_queue.get_nowait()
            except queue.Empty:
                break

    def stop_system(self):
        """Stop everything completely"""
        self.is_listening = False
        self.immediate_interrupt()
        pygame.mixer.quit()

    def start(self):
        """Start the system with complete process isolation"""
        self.listen_thread = threading.Thread(target=self.listen_continuously)
        self.process_thread = threading.Thread(target=self.process_text)
        self.speak_thread = threading.Thread(target=self.speak_responses)
        
        self.listen_thread.start()
        self.process_thread.start()
        self.speak_thread.start()
        
        try:
            while self.is_listening:
                time.sleep(0.1)
        except KeyboardInterrupt:
            self.stop_system()
        
        self.listen_thread.join()
        self.process_thread.join()
        self.speak_thread.join()

def main():
    system = VoiceChatSystem(lang='hi', model_name="llama3.2:1b")
    system.start()

if __name__ == "__main__":
    main()